# Get Point of Interest Data

In [ ]:
!pip install geopandas

In [2]:
# If you see the error `ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed` this can be ignored.
import os
import datetime as dt
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import shapely

## Download data

Download Point of Interest (POI) data, prepared by Geofabrik: http://download.geofabrik.de/africa/south-africa.html (also see the [different locations](http://download.geofabrik.de/index.html) available).

In [ ]:
!wget http://download.geofabrik.de/africa/south-africa-latest-free.shp.zip

In [ ]:
!mkdir -p south-africa-latest-free.shp
!unzip south-africa-latest-free.shp.zip -d south-africa-latest-free.shp

## Filter data down to our Area of Interest (AOI)

In [5]:
# read in shape from WKT file  with our AOI
WKT_FILE = 'Port_Elizabeth.wkt'

with open(WKT_FILE) as f:
    wkt = f.read()
    
shape = shapely.wkt.loads(wkt)

In [6]:
# Path to Geofabrik shp files folder
GEOFDIR = Path('south-africa-latest-free.shp/')

In [ ]:
# read in all POIs from file
gdf = geopandas.read_file(GEOFDIR / 'gis_osm_pois_free_1.shp')

In [8]:
# select only those inside our AOI
aoi_gdf = gdf[gdf.within(shape)].reset_index(drop=True)

In [ ]:
aoi_gdf.plot()
plt.title("POIs in the area of PE from OSM/Geofabrik")
plt.show()

In [ ]:
len(aoi_gdf) # amount of POIs in area 

In [ ]:
# top 20 categories available in PE
aoi_gdf['fclass'].value_counts().head(20)

In [12]:
# Save to geojson file we can use in MARS
res = Path("./GeoVectorBlueprint/Resources")
res.mkdir(parents=True, exist_ok=True)

# if the file already exists make a backup
if (res / 'pois.geojson').is_file():
    now = dt.datetime.now()
    os.rename(res / 'pois.geojson', res / f'bkp_pois_{now.strftime("%Y%m%d_%H%M%S")}.geojson')

aoi_gdf.to_file(res / "pois.geojson", driver="GeoJSON")